In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

val_split = 0.1
val_size = int(len(x_train) * val_split)

x_val, y_val = x_train[:val_size], y_train[:val_size]
x_train, y_train = x_train[val_size:], y_train[val_size:]

x_train, x_val, x_test = x_train / 255.0, x_val / 255.0, x_test / 255.0

x_train, y_train = torch.from_numpy(x_train).float(), torch.from_numpy(y_train).long()
x_val, y_val = torch.from_numpy(x_val).float(), torch.from_numpy(y_val).long()
x_test, y_test = torch.from_numpy(x_test).float(), torch.from_numpy(y_test).long()


class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=32, out_channels=3, kernel_size=3, stride=3, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=50, kernel_size=3, stride=1, padding=1)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(in_channels=50, out_channels=50, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(in_channels=50, out_channels=50, kernel_size=3, stride=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=1, stride=4)
        self.fc1 = nn.Linear(50 * 4 * 4, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x = F.pad(x, (1, 1, 1, 1), mode='constant', value=0)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.maxpool(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.maxpool2(x)
        batch_size = x.size(0)
        x = x.view(batch_size, -1)
        in_features = x.size(1)
        self.fc1 = nn.Linear(in_features, 100)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        optimizer = optim.SGD(model.parameters(), lr=0.01)
        optimizer.zero_grad()
        outputs = model(x_train)
        loss = F.cross_entropy(outputs, y_train)
        loss.backward()
        optimizer.step()
        return x

model = Model()

optimizer = optim.SGD(model.parameters(), lr=0.01)

# num_epochs not difned apprantly????
num_epochs = 10

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(x_train)
    loss = F.cross_entropy(outputs, y_train)
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        val_output = model(x_val)
        val_probs = F.softmax(val_output, dim=1)
        _, predicted = torch.max(val_probs, dim=1)

        correct = (predicted == y_val).sum().item()
        total = y_val.size(0)
        accuracy = (predicted == y_val).float().mean().item()
        print(f"Epoch {epoch+1}/{num_epochs}, Validation Accuracy: {accuracy * 100:.2f}%")

170498071/170498071 [==============================] - 4s 0us/step
